In [4]:
import open3d as o3d
import numpy as np
from roboflow import Roboflow
import cv2
from PIL import Image
import time
import colorsys
import os
# import pyransac3d as pyrsc

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
show_pcl = True

In [6]:
def visualizer(cloud):

    if type(cloud) != list:
        cloud = [cloud]
        
    if show_pcl:
        # o3d.visualization.draw_geometries(cloud,
        #                             zoom=0.57899999999999985,
        #                             front=  [ 0.053781796277127751, 0.99511863815317547, 0.082743062484869914 ],
        #                             up =  [ -0.99452345262671604, 0.045944145215534776, 0.093873519673282182 ],
        #                             lookat=[ 0.14950467828195535, -0.21448131248991498, 0.63221199653621662 ])
        
        center_of_mass = np.mean(np.asarray(cloud[0].points), axis=0)
        
        # print("com", center_of_mass)
        o3d.visualization.draw_geometries(cloud,
                                    zoom=0.7,
                                    front=[0, 1, 0],
                                    lookat=center_of_mass,
                                    up=[0, 0, 1])
    

In [7]:
def object_detection(img_path, model):

    prediction_data = model.predict(img_path, confidence=30, overlap=30).json() #50
    # change from 80, 30
    # visualize your prediction
    model.predict(img_path, confidence=30, overlap=30).save(img_path) #50
    return prediction_data

In [8]:
def get_selected_points(x_min, x_max, z_min, z_max, grid, data):
    selected_colors = []
    selected_points = []
    # img_test = cv2.imread('predictions\prediction22.jpg')

    for x in range(x_min, x_max, 1):
        for z in range(z_min, z_max, 1):
            idx = grid[x, z]
            if idx != -1:
                selected_colors.append(data[idx][0])
                selected_points.append(data[idx][1])
            # img_test[x, z] = [255, 0, 0] #BRG

    return np.array(selected_points), np.array(selected_colors)

In [9]:
def read_pcd(path, show=True):
    pcd = o3d.io.read_point_cloud(path)

    if show:
        visualizer(pcd)

    return pcd

In [10]:
def get_bounding_box(prediction_data):

    # 2D bounding box coordinates
    b_boxes = []

    for pred in prediction_data['predictions']:
        bounding_box = {
            'z': pred['x'],
            'x': pred['y'],
            'width': pred['width'],
            'height': pred['height'],
        }

        # Image dimensions (to scale 2D to 3D)
        # image_width = prediction_data['image']['width']
        # image_height = prediction_data['image']['height']

        # Calculate 3D coordinates (assuming z-coordinate is arbitrary)
        x_min = int(bounding_box['x'] - bounding_box['height']/2)
        z_min = int(bounding_box['z'] - bounding_box['width']/2)
        x_max = int(bounding_box['x']  + bounding_box['height']/2)
        z_max = int(bounding_box['z'] + bounding_box['width']/2)

        temp_bb = [x_min, x_max, z_min, z_max]
        b_boxes.append(temp_bb)

    return b_boxes

In [11]:
def draw_3d_bounding_box(pcd, all_pcds):

    for cld in all_pcds:

        bbox = cld.get_oriented_bounding_box()
        # both are the same
        # bbox = cld.get_axis_aligned_bounding_box()
        cld.paint_uniform_color((255, 0, 0))

        visualizer([cld, pcd, bbox])

        pcd = pcd.crop(bbox)

        visualizer(pcd)

    o3d.io.write_point_cloud(f"final_pcd.ply", pcd)

    # visualizer([pcd])

In [12]:
def algorithm(points, colors, shape1, shape2, resolution, reverse, axis_s, axes):
    check_grid = np.zeros((shape1+2*resolution, shape2+2*resolution), dtype=int) #They will set to 1 if that [x, y] grid is used; 0 -> false
    grid = np.ones((shape1+2*resolution, shape2+2*resolution)) * -1 #the actual grid which will have indexes mapped to a map
    img = np.ones((shape1+2*resolution, shape2+2*resolution, 3), dtype=np.uint8) * 170  # Gray out the background for better identification
    data = {} #{-1:[('RGB'), ('XYZ')], } #the data. key is index number saved into grid and values are the RGB and xyz data

    # axis_s is the axis along which we are sorting the points
    points_sort = points[np.argsort(points[:, axis_s])[::reverse]]
    colors_sort = colors[np.argsort(points[:, axis_s])[::reverse]]

    # Min and max values of the axes [X, Y, Z] (Doesn't depend on what axis we are doing it along)
    max_val = np.max(np.asarray(points_sort), axis=0)
    min_val = np.min(np.asarray(points_sort), axis=0)

    # The 2 coordinate axes for the 2d image
    axis1, axis2 = axes

    center1 = (min_val[axis1]+max_val[axis1])/2
    range1 = np.abs(min_val[axis1]) + np.abs(max_val[axis1])
    x_r_new = shape1 - 1

    center2 = (min_val[axis2]+max_val[axis2])/2
    range2 = np.abs(min_val[axis2]) + np.abs(max_val[axis2])
    z_r_new = shape2 - 1

    # For every single sorted point, iterate until you go to the end point
    for idx, val in enumerate(points_sort):
        # Conversion from 3D (but only two axes) to 2D

        # Scaling bt -ve inf. -> 1
        scale1 = ((val[axis1] - center1) / range1)
        scale2 = ((val[axis2] - center2) / range2)

        # print(scale1, scale2)
        index1 = int(scale1 * x_r_new + x_r_new/2) + resolution
        index2 = int(scale2 * z_r_new + z_r_new/2) + resolution

        if not check_grid[index1, index2]:

            check_grid[index1, index2] = 1
            data[idx] = [colors_sort[idx], val]
            grid[index1, index2] = idx
            col = np.uint8(colors_sort[idx]*255)
            img[index1, index2] = col
            # do around the point too of certain threshold

            dx_l = np.linspace(-resolution+index1, resolution+index1, 2*resolution+1, dtype=int)
            dy_l = np.linspace(-resolution+index2, resolution+index2, 2*resolution+1, dtype=int)

            dx, dy = np.meshgrid(dx_l, dy_l)

            temp_check_grid = check_grid[min(dx_l):max(dx_l)+1, min(dy_l):max(dy_l)+1]

            dx1 = (dx >= 0) & (dx < shape1) & ~temp_check_grid
            dy1 = (dy >= 0) & (dy < shape2) & ~temp_check_grid
            dx_dy = dx1 & dy1

            ones = np.argwhere(dx_dy)
            # scaling up
            ones[:, 0] += min(dx_l)
            ones[:, 1] += min(dy_l) #indices for check_grid

            # Set all the indices in check_grid to seen
            check_grid[ones[:, 0], ones[:, 1]] = 1
            img[ones[:, 0], ones[:, 1]] = col
    return check_grid, grid, img, data

In [13]:
def save_img(image, img_path):
    image = Image.fromarray(image)
    image.save(img_path)
    print("Image Saved!")

In [14]:
def get_bounding_box_2d(img_path, rect_threshold):
    thresholded_image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    edged = cv2.Canny(thresholded_image, 50, 100)

    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    bounding_boxes = []
    for contour in contours:
        # Get bounding box coordinates
        x, y, w, h = cv2.boundingRect(contour)
        if (w*h) > rect_threshold:
            bounding_boxes.append((x, y, w, h))

    original_image = cv2.imread(img_path)  # Load your original image here
    bounding_box_image = original_image.copy()
    for (x, y, w, h) in bounding_boxes:
        cv2.rectangle(bounding_box_image, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Draw green rectangles

    # Display or save the image with bounding boxes
    cv2.imwrite(img_path, bounding_box_image)

    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # Optionally, you can print the bounding boxes
    # print("Bounding boxes:", bounding_boxes)
    # x, y is top left point, not center
    # print(bounding_boxes)
    return bounding_boxes #[[x, y, w, h]]

In [15]:
def color_detection(img_path):
    image = cv2.imread(img_path)

    img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower_range = np.array([150, 160, 185])
    upper_range = np.array([360, 255, 255])

    mask = cv2.inRange(img_hsv, lower_range, upper_range)

    outside_range_mask = cv2.bitwise_not(mask)

    img_hsv[outside_range_mask != 0] = [0, 0, 255]

    result_image = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR)
    cv2.imwrite(img_path , result_image)

    bounding_boxes = get_bounding_box_2d(img_path, rect_threshold=2500)
    
    # x, y is top left point, not center
    # return the maxes and mins of both the axes
    axes_ranges = []
    for x, y, w, h in bounding_boxes: #[x, y, w, h]
        z_min = int(y)
        x_min = int(x)
        z_max = int(y + h)
        x_max = int(x + w)

        # print( x, y, w, h, x_min, x_max, z_min, z_max)

        axes_ranges.append([x_min, x_max, z_min, z_max])

    return axes_ranges


In [16]:
def initialization(pcd):
    x_shape = 7000
    y_shape = 7000
    z_shape = 7000

    iterations = [
        {"shape": [y_shape, z_shape], "reverse": 1, "axis":0, "axes": [1, 2]}, #X
        {"shape": [x_shape, z_shape], "reverse": 1, "axis":1, "axes": [0, 2]}, #Y
        {"shape": [x_shape, y_shape], "reverse": 1, "axis":2, "axes": [0, 1]}, #Z
        {"shape": [y_shape, z_shape], "reverse": -1, "axis":0, "axes": [1, 2]}, #-X
        {"shape": [x_shape, z_shape], "reverse": -1, "axis":1, "axes": [0, 2]}, #-Y
        {"shape": [x_shape, y_shape], "reverse": -1, "axis":2, "axes": [0, 1]} #-Z
    ]

    mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=pcd.get_center())

    visualizer([pcd, mesh_frame])

    rf = Roboflow(api_key="ts8jw999gCMVw72LHUzT")
    # project = rf.workspace().project("strawberry-flower-detection-il6jq")
    project = rf.workspace().project("polinizador")
    model = project.version(1).model

    j = 0

    return iterations, pcd, model, j

In [17]:
def threshold_bins(pcd):    
    threshold = 2 # 0 -> 1

    x, y, z = np.asarray(pcd.points)[:, :3].T

    std = np.std((x, y, z), axis=1) * threshold

    hist_x, bins_x = np.histogram(x, bins=50)
    hist_y, bins_y = np.histogram(y, bins=50)

    max_freq_bin_x = bins_x[np.argmax(hist_x)]
    max_freq_bin_y = bins_y[np.argmax(hist_y)]

    min_bound = (max_freq_bin_x, max_freq_bin_y, -2) - std
    max_bound = (max_freq_bin_x, max_freq_bin_y, 2) + std

    bounding_box = o3d.geometry.AxisAlignedBoundingBox(min_bound=min_bound,
                                                        max_bound=max_bound)
    pcd = pcd.crop(bounding_box)

    visualizer(pcd)

In [18]:
def threshold(pcd):
    threshold = 15 # in % max, min: [1.9655249  2.17742805 1.19400877], [-1.96551809e+00 -2.17745111e+00  9.51918373e-04]
    # lower is more threshold
    center_pc = pcd.get_center()
    max_v = pcd.get_max_bound()
    min_v = pcd.get_min_bound()

    size = (max_v - min_v) * 0.01 * threshold

    min_bound = center_pc - size
    max_bound = center_pc + size
    min_bound[2] = -2
    max_bound[2] = 2
    bounding_box = o3d.geometry.AxisAlignedBoundingBox(min_bound=min_bound,
                                                        max_bound=max_bound)

    pcd = pcd.crop(bounding_box)

    points = np.asarray(pcd.points)
    colors = np.asarray(pcd.colors)

    # sphere = pyrsc.Plane()
    # # s = 3
    # equation, best_inliers_plane = sphere.fit(np.asarray(pcd.points), thresh=0.03, maxIteration=80)
    
    # inlier_cloud_plane = pcd.select_by_index(best_inliers_plane)
    # outlier_cloud_plane = pcd.select_by_index(best_inliers_plane, invert=True)
    # pcd = outlier_cloud_plane
    # points = np.asarray(pcd.points)
    # colors = np.asarray(pcd.colors)

    visualizer(pcd)
    
    return points, colors

In [19]:
def main(iterations, points, colors, model, j, method="object", folder="test"):
    all_point_clouds = []
    
    folder_path = f"{folder}/numpy"
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    if not os.path.exists(f"{folder}/pcd_indv"):
        os.makedirs(f"{folder}/pcd_indv")
        
    for i in iterations:

        print("In iter", j)
        img_path=f"{folder_path}/img_{j}.jpg"
        
        start = time.time()
        check_grid, grid, img, data = algorithm(
            points=points,
            colors=colors,
            shape1=i["shape"][0],
            shape2=i["shape"][1],
            resolution=10,
            reverse=i["reverse"],
            axis_s=i["axis"],
            axes=i["axes"],)
        
        print(f"time taken: {int((time.time() - start))} seconds")
        save_img(image=img, img_path=img_path)

        if method == "object":
            predictions = object_detection(img_path=img_path, model=model)

            if len(predictions['predictions']) == 0:
                print("No objects found")
                j+=1
                continue
            
            print(f"{len(predictions['predictions'])} objects found")
            bounding_boxes = get_bounding_box(prediction_data=predictions)

            temp_points = []
            temp_colors = []

        elif method == "color":
            bounding_boxes = color_detection(img_path=img_path)
            temp_points = []
            temp_colors = []

        else:
            j += 1
            continue

        for x_min, x_max, z_min, z_max in bounding_boxes:

            selected_points, selected_colors = get_selected_points(x_min, x_max, z_min, z_max, grid, data)

            if not selected_points.shape == (0,):
                if type(temp_points) == list:
                    temp_points = selected_points
                    temp_colors = selected_colors

                temp_points = np.concatenate([temp_points, selected_points], axis=0)
                temp_colors = np.concatenate([temp_colors, selected_colors], axis=0)

        point_cloud = o3d.geometry.PointCloud()
        point_cloud.points = o3d.utility.Vector3dVector(temp_points)
        point_cloud.colors = o3d.utility.Vector3dVector(temp_colors)

        all_point_clouds.append(point_cloud)

        if show_pcl:
            visualizer([point_cloud])

        o3d.io.write_point_cloud(f"{folder}/pcd_indv/pcd_{j}.ply", point_cloud)

        j += 1

    if show_pcl:
        print("Showing all the detections")
        visualizer(all_point_clouds)

    pcd_final = o3d.geometry.PointCloud()

    for i in all_point_clouds:
        pcd_final += i

    o3d.io.write_point_cloud(f"{folder}/pcd_allpcd.ply", pcd_final)
    print("All pcds saved")

    return all_point_clouds, pcd_final

In [20]:
def filter_green(hsv_color):

    low_reject_h = 60/360
    high_reject_h = 170/360
 
    low_reject_s = 1
    high_reject_s = 1.0
 
    low_reject_v = 0.0
    high_reject_v = 1

    # if (low_reject_v <= hsv_color[2] <= high_reject_v):
    #     return True
    if (low_reject_h <= hsv_color[0] <= high_reject_h) and (low_reject_s <= hsv_color[1] <= high_reject_s) and (low_reject_v <= hsv_color[2] <= high_reject_v):
        return True
    else:
        return False

In [21]:
def filter_colors(hsv_colors):

    low_reject_h = 60/360
    high_reject_h = 170/360
 
    low_reject_s = 1
    high_reject_s = 1.0
 
    low_reject_v = 0.0
    high_reject_v = 1

    nT = 0

    for hsv_color in hsv_colors:
 
        if (low_reject_v <= hsv_color[2] <= high_reject_v):
            nT += 1
        if (low_reject_h <= hsv_color[0] <= high_reject_h) and (low_reject_s <= hsv_color[1] <= high_reject_s):
            nT += 1
        else:
            pass
    if nT >= len(hsv_colors)-nT:
        return True
    return False

In [22]:
def DBClustering(all_pcd):
    color = [
        [255, 0, 0],   # Red
        [0, 255, 0],   # Green
        [0, 0, 255],   # Blue
        [255, 255, 0], # Yellow
        [255, 0, 255], # Magenta
        [0, 255, 255], # Cyan
        [0, 128, 0],   # Green (dark)
        [0, 0, 128],   # Navy
        [128, 128, 0],  # Olive
        [128, 0, 128],  # Purple
        [0, 128, 128],  # Teal
        [128, 128, 128],# Gray
        [255, 165, 0],  # Orange
        [255, 192, 203],# Pink
        [0, 0, 0],      # Black
        [128, 0, 0],    # Maroon (dark)
        [0, 128, 0],    # Green (medium)
        [0, 128, 128],
        [128, 128, 128]
    ]

    with o3d.utility.VerbosityContextManager(
            o3d.utility.VerbosityLevel.Debug) as cm:
        labels = np.array(
            all_pcd.cluster_dbscan(eps=0.01, min_points=30))

    max_label = labels.max()
    outlier_cloud_sph_2 =[]
    bounding_boxes = []
    final_pcd = []
    l = 0
    pcd_bb = []
    all_segments = o3d.geometry.PointCloud()

    # Label -1 is all the points which didn't fit any of the clusters
    for i in range(0, max_label+1):
        indexes = np.argwhere(labels == i).flatten()
        new_pc = o3d.geometry.PointCloud()
        new_pc.points = o3d.utility.Vector3dVector(np.asarray(all_pcd.points)[indexes])
        new_pc.colors = o3d.utility.Vector3dVector(np.asarray(all_pcd.colors)[indexes])
        if l >= 19:
            l = 0
        # new_pc.paint_uniform_color(color[l])
        l+= 1
        # get the avg distance bt points and remove it if its above a certain threshold
        temp_dist = np.average(new_pc.compute_nearest_neighbor_distance())

        avg_color = np.average(new_pc.colors, axis=0)
        
        bb = new_pc.get_oriented_bounding_box()

        # Check for volume and density
        # if len(np.asarray(new_pc.points)) != 0 and not filter_green(avg_color):
        density = len(np.asarray(new_pc.points)) / bb.volume()

        # if density < 8387375.83954427:
        # if density > 900000 and density < 4700000:
        # if len(np.asarray(new_pc.points)) >= 100 and filter_colors(new_pc.colors):
        outlier_cloud_sph_2.append(new_pc)

        # find bounding box
        bounding_boxes.append(bb)
        temp_pcd = all_pcd.crop(bb)
        all_segments += temp_pcd
        final_pcd.append(temp_pcd)
        pcd_bb.append(temp_pcd)
        pcd_bb.append(bb)
        # final_pcd.append(bb)
        # visualizer(new_pc)
    # visualizer(pcd_bb)
    
    return bounding_boxes, all_segments

In [23]:
def write_bb(plant_id, bounding_boxes):
    dat = []
    for b in bounding_boxes:
        # print(b.center, b.R, b.extent)
        # print("\n")
        dat.append([b.center, b.R[0], b.R[1], b.R[2], b.extent])
    # print(np.asarray(dat))
    np.savez(f'{plant_id}_arrays.npz', np.asarray(dat))

In [24]:
def convert_rgb_2_hsv(all_rgb_colors):
    all_hsv_colors = []
 
    for i in range(len(all_rgb_colors)):
        temp_color = all_rgb_colors[i]
        temp = colorsys.rgb_to_hsv(temp_color[0], temp_color[1], temp_color[2])
        all_hsv_colors.append(temp)
 
    all_hsv_colors = np.asarray(all_hsv_colors)
 
    return all_hsv_colors

In [25]:
def hsv_filter_pistil(hsv_colors):
    hsv_colors = convert_rgb_2_hsv(hsv_colors)
    # Only get pistil
    low_h = 40/360
    high_h = 63/360

    low_s = 0.30
    high_s = 1.0

    low_v = 0.65
    high_v = 1.0
    nT = 0

    for hsv_color in hsv_colors:
        if (low_h <= hsv_color[0] <= high_h) and (low_s <= hsv_color[1] <= high_s) and (low_v <= hsv_color[2] <= high_v):
            # return True
            nT += 1
    # print(nT, len(hsv_colors-nT))
    if nT > 10:
        return True
    return False

In [26]:
def hsv_filter_white(hsv_color): #flower
    
    # Our red strawberry flowers have a very particular color of petal
    # We will segment the flowers out based on this, firstly
    # --
    # H,S,V = {(300, 0] U [0, 60), (0.3, 1), (0.3, 1)}

    low_h1 = 0/180
    high_h1 = 180/180

    low_s = 0/255
    high_s = 100/255

    low_v = 0/255
    high_v = 255/255
    
    # or low_h2 <= hsv_color[0] <= high_h2)

    if (low_h1 <= hsv_color[0] <= high_h1) and low_s <= hsv_color[1] <= high_s and low_v <= hsv_color[2] <= high_v:
        return True
    else:
        return False

In [27]:
def test_dbscan(pcd_final):
    color = [
        [255, 0, 0],   # Red
        [0, 255, 0],   # Green
        [0, 0, 255],   # Blue
        [255, 255, 0], # Yellow
        [255, 0, 255], # Magenta
        [0, 255, 255], # Cyan
        [0, 128, 0],   # Green (dark)
        [0, 0, 128],   # Navy
        [128, 128, 0],  # Olive
        [128, 0, 128],  # Purple
        [0, 128, 128],  # Teal
        [128, 128, 128],# Gray
        [255, 165, 0],  # Orange
        [255, 192, 203],# Pink
        [0, 0, 0],      # Black
        [128, 0, 0],    # Maroon (dark)
        [0, 128, 0],    # Green (medium)
        [0, 128, 128],
        [128, 128, 128]
    ]

    with o3d.utility.VerbosityContextManager(
            o3d.utility.VerbosityLevel.Debug) as cm:
        labels = np.array(
            pcd_final.cluster_dbscan(eps=0.01, min_points=20))

    max_label = labels.max()
    outlier_cloud_sph_2 =[]
    bounding_boxes = []
    final_pcd = []
    l = 0
    pcd_bb = []
    # outlier_cloud_sph_2.append(pcd)
    all_segments = o3d.geometry.PointCloud()

    # Label -1 is all the points which didn't fit any of the clusters
    nu = 0
    for i in range(0, max_label+1):
        indexes = np.argwhere(labels == i).flatten()
        new_pc = o3d.geometry.PointCloud()
        new_pc.points = o3d.utility.Vector3dVector(np.asarray(pcd_final.points)[indexes])
        new_pc.colors = o3d.utility.Vector3dVector(np.asarray(pcd_final.colors)[indexes])
        if l >= 19:
            l = 0
        # new_pc.paint_uniform_color(color[l])
        l+= 1
        # get the avg distance bt points and remove it if its above a certain threshold
        temp_dist = np.average(new_pc.compute_nearest_neighbor_distance())

        avg_color = np.average(new_pc.colors, axis=0)

        bb = new_pc.get_oriented_bounding_box()
        # print(np.asarray(bb.extent))
        w, h, le = np.asarray(bb.extent)
        # if len(np.asarray(new_pc.points)) != 0 and not filter_green(avg_color):
        volume = float(bb.volume())
        # if density < 8387375.83954427:
        # if filter_colors(new_pc.colors) and volume > 2 * np.power(10.0, -5):
        # if hsv_filter_pistil(new_pc.colors) and volume > 2 * np.power(10.0, -5):
        # if le > 0.006:
        # print(i)
        # print(w, h, le)
        # if hsv_filter_pistil(new_pc.colors):
        # if True:
        cnt = 0
        rgb_colors = convert_rgb_2_hsv(new_pc.colors)
        for col in rgb_colors:
            # if hsv_filter(col):
            if hsv_filter_white(col):
                cnt += 1
        # print("cluster", i, cnt, len(new_pc.colors))
        # avg_rgb = np.mean(new_pc.colors, axis=0)
        # avg_hsv = colorsys.rgb_to_hsv(avg_rgb[0], avg_rgb[1], avg_rgb[2])
        # print("avg_hsv:", avg_hsv)
        # print("sdsd", hsv_filter(avg_hsv))
        # if hsv_filter(avg_hsv):
        if cnt > len(new_pc.colors)*0.50:
            # if len(np.asarray(new_pc.points)) >= 100 and filter_colors(new_pc.colors):
            outlier_cloud_sph_2.append(new_pc)
            # find bounding box
            bounding_boxes.append(bb)
            temp_pcd = pcd_final.crop(bb)
            all_segments += temp_pcd
            final_pcd.append(temp_pcd)
            pcd_bb.append(temp_pcd)
            pcd_bb.append(bb)
            # final_pcd.append(bb)
            # visualizer(temp_pcd)
        nu += 1
    visualizer(pcd_bb)
    # o3d.io.write_point_cloud(f"temp/{folder}/pcd_DB.ply", all_segments)
    # write_bb(idx, bounding_boxes)
    return bounding_boxes, all_segments

In [28]:
import matplotlib

In [29]:
# Maybe try this
def filter_white_yellow_flowers(pcd):
    colors = np.asarray(pcd.colors)
    hsv_colors = matplotlib.colors.rgb_to_hsv(colors)
    
    mask = np.array([hsv_filter_white_yellow(color) for color in hsv_colors])
    
    filtered_pcd = pcd.select_by_index(np.where(mask)[0])
    return filtered_pcd


In [30]:
import time

def run_indv(path, path_folder, folder):

    start_time = time.time()
    
    pcd = o3d.io.read_point_cloud(path)

    iterations, pcd, model, j = initialization(pcd)

    points, colors = threshold(pcd=pcd)

    all_point_clouds, pcd_final = main(iterations=iterations,
        points=points,
        colors=colors,
        model=model,
        j=j,
        folder=path_folder+folder)

    # bounding_boxes, all_segments, pcd_bb, final_pcd = DBClustering(
    #     all_pcd=pcd_final,
    #     pcd=pcd)

    # pcd_final_all_pth = f"./temp/{folder}/pcd_allpcd.ply"
    # pcd_final_all = o3d.io.read_point_cloud(pcd_final_all_pth)

    bounding_boxes, all_segments = test_dbscan(pcd_final)

    o3d.io.write_point_cloud(f"{path_folder}/{folder}/pcd_DB.ply", all_segments)
    end_time = time.time()
    total_time = end_time - start_time
    print("total time:", total_time)    
    return bounding_boxes


In [ ]:
path = '../3dmodels/Old Final Data/IphoneData.ply'
path_folder = 'C:/Users/harsh/OneDrive - Georgia Institute of Technology/Documents/GTRI/Documents/test_code/deeplearning/temp/'
folder = "Iphone_new1"
bb = run_indv(path, path_folder, folder)
write_bb(folder, bb)

In [50]:
write_bb(folder, bb)

In [49]:
def write_bb(plant_id, bounding_boxes):
    dat = {}
    for i, b in enumerate(bounding_boxes):
        dat[f'box_{i}'] = {
            'center': b.center,
            'R': b.R,
            'extent': b.extent
        }
    np.savez(f'C:/Users/harsh/OneDrive - Georgia Institute of Technology/Documents/GTRI/Documents/test_code/deeplearning/BB_Data/{plant_id}_arrays.npz', **dat)

In [32]:
def run_all_pcd():
    directory_path = 'C:/Users/harsh/OneDrive - Georgia Institute of Technology/Documents/GTRI/Documents/test_code/3dmodels/Final Data'
    path_folder = 'C:/Users/harsh/OneDrive - Georgia Institute of Technology/Documents/GTRI/Documents/test_code/deeplearning/temp/'

    for f in os.listdir(directory_path):
        folder = f[:-4]
        print(os.path.join(directory_path, f))
        path_pcd = os.path.join(directory_path, f)
        print("Doing file:", folder)
        run_indv(path_pcd, path_folder, folder)
        # break
# run_all_pcd()

In [33]:
# The process of applying statistical outlier removal and then DBScan is giving much better results. I just have to find a way to remove those small leaf clusters.

In [34]:
def run_db_only():
    directory_path = 'C:/Users/harsh/OneDrive - Georgia Institute of Technology/Documents/GTRI/Documents/test_code/deeplearning/temp/'
    path_folder = 'C:/Users/harsh/OneDrive - Georgia Institute of Technology/Documents/GTRI/Documents/test_code/deeplearning/temp/'

    for f in os.listdir(directory_path):
        if f == "Iphone_new":
            folder = f
            print(os.path.join(directory_path, f))
            # path_pcd = os.path.join(directory_path, f)
            pcd_file = os.path.join(path_folder, folder, "pcd_allpcd.ply")
            pcd = o3d.io.read_point_cloud(pcd_file)
            print("Doing file:", folder)
            segments = statistical_outlier_removal(pcd)
            visualizer(segments)
            segments = remove_outliers(segments)
            bounding_boxes, all_segments = test_dbscan(segments)
            o3d.io.write_point_cloud(os.path.join(path_folder, folder, "pcd_DB_3.ply"), all_segments)
            
            pcd_file = os.path.join(path_folder, folder, "pcd_DB_3.ply")
            pcd = o3d.io.read_point_cloud(pcd_file)
            print("Doing file:", folder)
            segments = statistical_outlier_removal(pcd)
            visualizer(segments)
            segments = remove_outliers(segments)
            # bounding_boxes, all_segments = test_dbscan(segments)
            o3d.io.write_point_cloud(os.path.join(path_folder, folder, "pcd_DB_3.ply"), all_segments)
            # break

In [35]:
# run_db_only()

In [36]:
def write_bb(plant_id, bounding_boxes):
    dat = {}
    for i, b in enumerate(bounding_boxes):
        dat[f'box_{i}'] = {
            'center': b.center,
            'R': b.R,
            'extent': b.extent
        }
    np.savez(f'C:/Users/harsh/OneDrive - Georgia Institute of Technology/Documents/GTRI/Documents/test_code/deeplearning/BB_Data/{plant_id}_arrays.npz', **dat)

In [37]:
def filter_pcd_final1(pcd):
    def statistical_outlier_removal(pcd, nb_neighbors=100, std_ratio=0.5):
        _, ind = pcd.remove_statistical_outlier(nb_neighbors, std_ratio)
        return pcd.select_by_index(ind)

    def remove_outliers(pcd, nb_points=15, radius=0.01):
        # cl, ind = pcd.remove_radius_outlier(nb_neighbors)
        _ , ind = pcd.remove_radius_outlier(nb_points, radius)
        return pcd.select_by_index(ind)

    # pcd_file = 'C:/Users/harsh/OneDrive - Georgia Institute of Technology/Documents/GTRI/Documents/test_code/deeplearning/temp/Data6/pcd_allpcd.ply'
    # pcd = o3d.io.read_point_cloud(pcd_file)
    # avg_rgb = np.mean(pcd.colors, axis=0)
    # avg_hsv = colorsys.rgb_to_hsv(avg_rgb[0], avg_rgb[1], avg_rgb[2])
    # print("avg_hsv:", avg_hsv)
    # print("sdsd", hsv_filter(avg_hsv))
    segments = statistical_outlier_removal(pcd)
    visualizer(segments)
    segments = remove_outliers(segments)
    visualizer(segments)
    bb, cluster_pcd = test_dbscan(segments)
    
    return bb, cluster_pcd

directory_path = 'C:/Users/harsh/OneDrive - Georgia Institute of Technology/Documents/GTRI/Documents/test_code/deeplearning/temp/'
path_folder = 'C:/Users/harsh/OneDrive - Georgia Institute of Technology/Documents/GTRI/Documents/test_code/deeplearning/temp/'

for f in os.listdir(directory_path):
    # if f == "Iphone_new":
    folder = f
    print(os.path.join(directory_path, f))
    # path_pcd = os.path.join(directory_path, f)
    pcd_file = os.path.join(path_folder, folder, "pcd_allpcd.ply")
    pcd = o3d.io.read_point_cloud(pcd_file)
    bb, cluster_pcd = filter_pcd_final1(pcd)
    # o3d.io.write_point_cloud(os.path.join(path_folder, folder, "pcd_DB_4.ply"), cluster_pcd)
    write_bb(f, bb)


C:/Users/harsh/OneDrive - Georgia Institute of Technology/Documents/GTRI/Documents/test_code/deeplearning/temp/Data10
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 15
C:/Users/harsh/OneDrive - Georgia Institute of Technology/Documents/GTRI/Documents/test_code/deeplearning/temp/Data4
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 11
C:/Users/harsh/OneDrive - Georgia Institute of Technology/Documents/GTRI/Documents/test_code/deeplearning/temp/Data5
[Open3D DEBUG] Precompute neighbors.

KeyboardInterrupt: 

In [ ]:
def hsv_filter(hsv_color): #flower
    
    # Our red strawberry flowers have a very particular color of petal
    # We will segment the flowers out based on this, firstly
    # --
    # H,S,V = {(300, 0] U [0, 60), (0.3, 1), (0.3, 1)}

    low_h1 = 0/360
    high_h1 = 40/360
    low_h2 = 80/360
    high_h2 = 360/360

    low_s = 0
    high_s = 0.8

    low_v = 0
    high_v = 1


    if (low_h1 <= hsv_color[0] <= high_h1 or low_h2 <= hsv_color[0] <= high_h2) and low_s <= hsv_color[1] <= high_s and low_v <= hsv_color[2] <= high_v:
        return False
    else:
        return True